# A123 18650 LIB State-of-Charge Prediction System

In [1]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image, ImageTk
import ctypes

# Load the trained model
model = load_model('30deg.h5')

# Define the min and max values for normalization
input_min = np.array([-3.849407,2.409066,31.174595]) 
input_max = np.array([2.061073,3.592155,33.004131])  

def minmax_normalization(x, min_val, max_val):
    return (x - min_val) / (max_val - min_val)

def show_custom_message(title, message):
    custom_messagebox = tk.Toplevel()
    custom_messagebox.title(title)
    custom_messagebox.configure(bg="grey")

    # Centering the message box on the screen
    windowWidth = custom_messagebox.winfo_reqwidth()
    windowHeight = custom_messagebox.winfo_reqheight()
    positionRight = int(custom_messagebox.winfo_screenwidth() / 2 - windowWidth / 2)
    positionDown = int(custom_messagebox.winfo_screenheight() / 2 - windowHeight / 2)
    custom_messagebox.geometry("+{}+{}".format(positionRight, positionDown))

    label = tk.Label(custom_messagebox, text=message, bg="grey",font=("Helvetica", 12, "bold"),fg="white")
    label.pack(padx=20, pady=10)

    ok_button = tk.Button(custom_messagebox, text="OK",font=("Helvetica", 12, "bold"), command=custom_messagebox.destroy, bg="green",fg="white")
    ok_button.pack(pady=5)

    custom_messagebox.mainloop()

def predict_output():
    try:
        # Get user input values
        input1 = float(entry1.get())
        input2 = float(entry2.get())
        input3 = float(entry3.get())
        
        if not (-3.8 <= input1 <= 2):
            raise ValueError("Input Current must be within the range -3.8 to 2.")
        if not (2.4 <= input2 <= 3.6):
            raise ValueError("Input Voltage must be within the range 2.4 to 3.6.")
        if not (31 <= input3 <= 33):
            raise ValueError("Input Temperature must be within the range 31 to 33.")

        # Normalize the input data using min-max normalization
        input_data = np.array([input1, input2, input3])
        input_data = minmax_normalization(input_data, input_min, input_max)
        input_data = input_data.reshape(1, 3)

        # Make prediction using the model
        prediction = model.predict(input_data)[0][0][0]

        # Determine the status based on the input current
        status = ""
        if input1 < 0:
            status = "Discharging"
        elif input1 > 0:
            status = "Charging"
        else:
            status = "Idle"

        # Display the predicted output
        show_custom_message("State-of-Charge Prediction", f"The State-of-Charge is: {int(np.round(prediction))} %\nStatus: {status}")

    except ValueError as ve:
        show_custom_message("Input Error", str(ve))

# Create the main application window
root = tk.Tk()
root.title("State-of-Charge Prediction 30Deg(C)")
root.configure(bg="grey")

# heading for the display
heading_label = tk.Label(root, text="State-of-Charge Prediction System\nA123 Li-Ion Battery", font=("Helvetica", 18, "bold"), bg="grey", fg="white")
heading_label.grid(row=1, column=0, columnspan=2, padx=10, pady=5, sticky="n")

# Load and resize the image
image = Image.open("D:/1.png")  # Replace "your_image.jpg" with the actual image file name
image = image.resize((150, 100))  # Resize the image as needed
photo = ImageTk.PhotoImage(image)

# Display the image using a Label widget
image_label = tk.Label(root, image=photo)
image_label.grid(row=2, column=0, columnspan=2, padx=10, pady=5)  # Define image_label in the scope where it's used

# content for the display
content_label = tk.Label(root, text="The SOC is defined as the available capacity of the battery, \nexpressed as a percentage of its rated capacity.\nInput Current: -3.8 ≤ Current ≤ 2\n Input Voltage: 2.4 ≤ Voltage ≤ 3.6\nInput Temperature: 31 ≤ Temperature ≤ 33", font=("Helvetica", 12,"bold"), bg="grey", fg="white")
content_label.grid(row=3, column=0, columnspan=2, padx=10, pady=5, sticky="n")

# Create input labels and entry fields
label1 = tk.Label(root, text="Current :", font=("Helvetica", 12, "bold"), bg="orange", fg="white", width=12, anchor="e")
label1.grid(row=4, column=0, padx=10, pady=5)
entry1 = tk.Entry(root, font=("Helvetica", 14))
entry1.grid(row=4, column=1, padx=10, pady=5, sticky="w")

label2 = tk.Label(root, text="Voltage :", font=("Helvetica", 12, "bold"), bg="red", fg="white", width=12, anchor="e")
label2.grid(row=5, column=0, padx=10, pady=5)
entry2 = tk.Entry(root, font=("Helvetica", 14))
entry2.grid(row=5, column=1, padx=10, pady=5, sticky="w")

label3 = tk.Label(root, text="Temperature :", font=("Helvetica", 13, "bold"), bg="blue", fg="white", width=12, anchor="e")
label3.grid(row=6, column=0, padx=10, pady=5)
entry3 = tk.Entry(root, font=("Helvetica", 14))
entry3.grid(row=6, column=1, padx=10, pady=5, sticky="w")

# Create predict button
predict_button = tk.Button(root, text="SOC", font=("Helvetica", 12, "bold"),command=predict_output, bg="green", fg="white")
predict_button.grid(row=7, column=0, columnspan=2, padx=10, pady=10)

# Add content
content_label = tk.Label(root, text="Created by Meikandadevan ", font=("Helvetica", 8,"bold"), bg="grey", fg="white")
content_label.grid(row=8, column=0, columnspan=2, padx=10, pady=5, sticky="n")

# Run the application
root.mainloop()


1/1 [==============================] - 0s 12ms/step
